In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Import libabries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import seaborn as sn
import pandas as pd
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import time
import copy

from PIL import Image
from tqdm import tqdm


In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
use_cuda = torch.cuda.is_available()


In [4]:
train_dir = '/kaggle/input/ima205-challenge-2024/Train/Train'
train_data = '/kaggle/input/ima205-challenge-2024/metadataTrain.csv'

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [6]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with ID and class labels.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        img_labels = pd.read_csv(csv_file)
        self.img_labels = img_labels.drop(columns=['SEX','AGE','POSITION'])
        self.img_labels['CLASS'] = self.img_labels['CLASS'] - 1

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f"{self.img_labels.iloc[idx, 0]}.jpg")
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label


In [7]:
dataset = CustomImageDataset(csv_file=train_data, img_dir=train_dir, transform=transform)


In [8]:
print(len(dataset))

18998


In [9]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [10]:
model = models.resnet50(pretrained=True)
model

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
num_features = model.fc.in_features 
print('Number of features from pre-trained model', num_features)

Number of features from pre-trained model 2048


In [12]:
model.fc = nn.Linear(num_features, 8)  # 8 classes as per your requirement
softmax = nn.Softmax(dim=1)



In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [14]:
model = model.to(device)

In [15]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        with tqdm(dataloader, unit="batch") as tepoch:
            for inputs, labels in tepoch:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                #print('a')
                loss = criterion(outputs, labels)

                # Backward + optimize
                loss.backward()
                optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'Epoch {epoch + 1}/{num_epochs} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Call the training function
model = train_model(model, dataloader, criterion, optimizer, num_epochs=18)


100%|██████████| 1188/1188 [07:06<00:00,  2.79batch/s]


Epoch 1/18 Loss: 1.1024 Acc: 0.6113


100%|██████████| 1188/1188 [04:57<00:00,  3.99batch/s]


Epoch 2/18 Loss: 1.0281 Acc: 0.6333


100%|██████████| 1188/1188 [04:52<00:00,  4.06batch/s]


...

Epoch 16/18 Loss: 0.8914 Acc: 0.6762


100%|██████████| 1188/1188 [04:48<00:00,  4.12batch/s]


Epoch 17/18 Loss: 0.9014 Acc: 0.6760


100%|██████████| 1188/1188 [04:45<00:00,  4.16batch/s]

Epoch 18/18 Loss: 0.8963 Acc: 0.6752


In [16]:
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_files = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]  # Return image and its ID

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [17]:
test_dataset = TestImageDataset(img_dir='/kaggle/input/ima205-challenge-2024/Test/Test', transform=transform)

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [18]:
model.eval()  # Set model to evaluation mode
predictions = []

with torch.no_grad():
    for image, image_id in test_dataloader:
        image = image.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_class = predicted.item() + 1  # Since we shifted labels to start from 0
        predictions.append((image_id[0], predicted_class))  # Append (image_id, predicted_class) tuple

# Write predictions to CSV file
submission_df = pd.DataFrame(predictions, columns=['ID', 'CLASS'])
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [20]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

output_dir = '/kaggle/working/'

# List files in the output directory
print("Files in output directory:")
print(os.listdir(output_dir))

# Provide a link to download the submission.csv file
print("\nDownload the submission.csv file:")
print(f"[submission.csv](./submission.csv)")

Files in output directory:
['.virtual_documents', 'submission.csv']

Download the submission.csv file:
[submission.csv](./submission.csv)
